In [2]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 12.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.1/318.1 kB 36.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 59.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 7.4 MB

In [3]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/ERA_V2/Assignment20/

Mounted at /content/drive
/content/drive/MyDrive/ERA_V2/Assignment20


In [19]:
import pickle

# Load vocab from the file
with open('vocab.pkl', 'rb') as f:
    loaded_vocab = pickle.load(f)

# Load vocab from the file
with open('merges.pkl', 'rb') as f:
    merges = pickle.load(f)

In [20]:
import gradio as gr
import sentencepiece as spm
sp = spm.SentencePieceProcessor()
sp.load('tok_ml.model')

True

In [21]:
def get_stats(ids):
    counts = {}
    for pair in zip(ids, ids[1:]):
        counts[pair] = counts.get(pair, 0) + 1
    return counts

def merge(ids, pair, idx):
  newids = []
  i = 0
  while i < len(ids):
    if i < len(ids) - 1 and ids[i] == pair[0] and ids[i+1] == pair[1]:
      newids.append(idx)
      i += 2
    else:
      newids.append(ids[i])
      i += 1
  return newids
def encode(text):
  # given a string, return list of integers (the tokens)
  tokens = list(text.encode("utf-8"))
  while len(tokens) >= 2:
    stats = get_stats(tokens)
    pair = min(stats, key=lambda p: merges.get(p, float("inf")))
    if pair not in merges:
      break # nothing else can be merged
    idx = merges[pair]
    tokens = merge(tokens, pair, idx)
  return tokens


def decode_r(ids_str):
    # Convert the comma-separated string of IDs to a list of integers
    ids = [int(x.strip()) for x in ids_str.split(",")]
    # Given ids (list of integers), return Python string
    tokens = b"".join(loaded_vocab[idx] for idx in ids)
    text = tokens.decode("utf-8", errors="replace")
    return text

def vocab(n):
  vocab = [[sp.id_to_piece(idx), idx] for idx in range(sp.get_piece_size())]
  return vocab[n:]


In [22]:
import numpy as np

enc = gr.Interface(
    fn=encode,
    inputs=gr.Textbox(lines=2, placeholder="Enter text"),
    examples=[ "ജോലിയില്‍ നിന്ന്",
              "ലൈഫ് ഇന്‍ഷുറന്‍സ് പ്രീമിയം"],
    outputs="text",
    title="Encode",
)

dec = gr.Interface(
    fn=decode_r,
    inputs=gr.Textbox(
        lines=2,
        placeholder="Enter a list of IDs separated by commas"
    ),
    examples=[
        "489, 542, 532, 168, 493, 258",
        "297, 348, 494, 608, 392, 183, 353, 552, 369, 543, 305, 291, 285",
        "297"
    ],
    outputs="text",
    title="Decode",
    description="Input a list of IDs separated by commas to decode into a string."
)

tok = gr.Interface(
    fn=vocab,
    inputs=gr.Number(label="Enter number between 0 to 649 to view token generated"),
    examples=[350], # list input
    outputs="text", # Added output type
    title="Token",
)

gr.TabbedInterface(
    [enc, dec, tok],
    ["Encode", "Decode", "Token"],
).launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://baae9d1b587cec78d2.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
